# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


Link dataset kaggle : https://www.kaggle.com/datasets/carrie1/ecommerce-data/data

# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [6]:
url='https://drive.google.com/uc?id=1l4OlQwoikH_mqFRJUZeJO64_pyr82oKg'
df = pd.read_csv(url, encoding='ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [7]:
print("Info Dataset:")
df.info()

print("\nStatistik Deskriptif:")
df.describe()

Info Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB

Statistik Deskriptif:


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [8]:
import pandas as pd

# --- 1. DATA CLEANING & PREPROCESSING ---
# Membersihkan data tidak valid, outlier, missing values, dan duplikat
df_clean = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)].copy()
df_clean = df_clean[df_clean['Quantity'] < 10000]
df_clean.dropna(subset=['CustomerID'], inplace=True)
df_clean = df_clean.drop_duplicates()
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

print(f"Ukuran data setelah cleaning: {df_clean.shape}")

# --- 2. FEATURE ENGINEERING (AGGREGATION) ---
# Mengelompokkan data penjualan berdasarkan produk dan periode bulan
df_clean['Month_Year'] = df_clean['InvoiceDate'].dt.to_period('M')

monthly_sales = df_clean.pivot_table(
    index='StockCode',
    columns='Month_Year',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

# --- 3. FEATURE EXTRACTION ---
# Menghitung metrik statistik (Mean, Std Dev, Max) sebagai fitur produk
product_features = pd.DataFrame()
product_features['Avg_Sales'] = monthly_sales.mean(axis=1)
product_features['Std_Dev'] = monthly_sales.std(axis=1)
product_features['Max_Sales'] = monthly_sales.max(axis=1)

# Menghitung Coefficient of Variation (CV) untuk mengukur stabilitas
product_features['CV'] = product_features['Std_Dev'] / product_features['Avg_Sales']
product_features['CV'] = product_features['CV'].fillna(0)

# --- 4. DATA JOINING & REVENUE ANALYSIS ---
# Menggabungkan data statistik dengan informasi produk dan menghitung rata-rata omzet
product_info = df_clean.groupby('StockCode').agg({'UnitPrice': 'mean', 'Description': 'first'})
final_data = product_features.join(product_info)
final_data['Avg_Revenue'] = final_data['Avg_Sales'] * final_data['UnitPrice']

# --- 5. THRESHOLD CALCULATION ---
# Menentukan ambang batas (Q3 dan Median) untuk pengelompokan kuadran
REV_Q3 = final_data['Avg_Revenue'].quantile(0.75)
MAX_Q3 = final_data['Max_Sales'].quantile(0.75)
CV_MEDIAN = final_data['CV'].median()

print(f"Threshold Omzet Tinggi (Revenue) > {REV_Q3:.2f}")
print(f"Threshold Stabil (CV) < {CV_MEDIAN:.2f}")

# --- 6. QUADRANT LABELING ---
# Fungsi untuk mengklasifikasikan produk ke dalam 4 kategori (0-3)
def quadrant_labeling(row):
    revenue = row['Avg_Revenue']
    max_sales = row['Max_Sales']
    volatility = row['CV']

    is_high_revenue = (revenue > REV_Q3) or (max_sales > MAX_Q3)
    is_stable = (volatility <= CV_MEDIAN)

    if is_high_revenue and is_stable:
        return 3
    elif is_high_revenue:
        return 2
    elif is_stable:
        return 1
    else:
        return 0

# Menerapkan label dan menyimpan hasil akhir ke CSV
final_data['Label'] = final_data.apply(quadrant_labeling, axis=1)

print("\nSebaran 4 Label Baru:")
print(final_data['Label'].value_counts().sort_index())
final_data.reset_index().to_csv('data_training_final.csv', index=False)


Ukuran data setelah cleaning: (392690, 8)
Threshold Omzet Tinggi (Revenue) > 172.19
Threshold Stabil (CV) < 1.39

Sebaran 4 Label Baru:
Label
0    1436
1     982
2     396
3     850
Name: count, dtype: int64


In [9]:
final_data.head()

,Avg_Sales,Std_Dev,Max_Sales,CV,UnitPrice,Description,Avg_Revenue,Label
StockCode,,,,,,,,
10002,63.307692,111.949680,337,1.768342,0.850000,INFLATABLE POLITICAL GLOBE,53.811538,0
10080,22.384615,31.473636,91,1.406039,0.411905,GROOVY CACTUS INFLATABLE,9.220330,0
10120,14.769231,15.812199,49,1.070618,0.210000,DOGGY RUBBER,3.101538,1
10123C,0.384615,1.120897,4,2.914332,0.650000,HEARTS WRAPPING TAPE,0.250000,0
10124A,1.230769,1.964427,5,1.596097,0.420000,SPOTS ON RED BOOKCOVER TAPE,0.516923,0
